<a href="https://colab.research.google.com/github/smdbg/colab/blob/main/Step_3_Vector_DB_open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import shutil
from typing import Dict, List, Tuple

# pip install lancedb docling openai tiktoken python-dotenv transformers

import lancedb
import tiktoken
from docling.chunking import HybridChunker
from docling.document_converter import DocumentConverter
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector
from openai import OpenAI
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

# --- CONFIGURATION ---
OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Initialize OpenAI client
client = OpenAI()

# --- CUSTOM TOKENIZER WRAPPER ---
class OpenAITokenizerWrapper(PreTrainedTokenizerBase):
    """Minimal wrapper for OpenAI's tokenizer to match HuggingFace interface."""

    def __init__(
        self, model_name: str = "cl100k_base", max_length: int = 8191, **kwargs
    ):
        super().__init__(model_max_length=max_length, **kwargs)
        self.tokenizer = tiktoken.get_encoding(model_name)
        self._vocab_size = self.tokenizer.max_token_value

    def tokenize(self, text: str, **kwargs) -> List[str]:
        return [str(t) for t in self.tokenizer.encode(text)]

    def _tokenize(self, text: str) -> List[str]:
        return self.tokenize(text)

    def _convert_token_to_id(self, token: str) -> int:
        return int(token)

    def _convert_id_to_token(self, index: int) -> str:
        return str(index)

    # --- FIX: Добавяме липсващия метод __len__ ---
    def __len__(self) -> int:
        return self._vocab_size

    def get_vocab(self) -> Dict[str, int]:
        return dict(enumerate(range(self.vocab_size)))

    @property
    def vocab_size(self) -> int:
        return self._vocab_size

    def save_vocabulary(self, *args) -> Tuple[str]:
        return ()

    @classmethod
    def from_pretrained(cls, *args, **kwargs):
        return cls()

# --- MAIN PIPELINE ---

# 1. Setup Tokenizer
# text-embedding-3-large поддържа 8191 токена
tokenizer = OpenAITokenizerWrapper(model_name="cl100k_base", max_length=8191)
MAX_TOKENS = 8191

# 2. Extract Data
print("⏳ Converting PDF...")
converter = DocumentConverter()
result = converter.convert("https://arxiv.org/pdf/2408.09869")

# 3. Chunking
print("🔪 Chunking...")
chunker = HybridChunker(
    tokenizer=tokenizer,
    max_tokens=MAX_TOKENS,
    merge_peers=True,
)

chunk_iter = chunker.chunk(dl_doc=result.document)
chunks = list(chunk_iter)
print(f"✅ Created {len(chunks)} chunks.")

# 4. LanceDB Setup
if os.path.exists("data/lancedb"):
    shutil.rmtree("data/lancedb")

db = lancedb.connect("data/lancedb")

# 5. OpenAI Embedding Function (Native)
# LanceDB автоматично ще използва ключа от os.environ
func = get_registry().get("openai").create(name="text-embedding-3-large")

# 6. Schema Definition
class ChunkMetadata(LanceModel):
    filename: str | None
    page_numbers: List[int] | None
    title: str | None

class Chunks(LanceModel):
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()
    metadata: ChunkMetadata

table = db.create_table("docling", schema=Chunks, mode="overwrite")

# 7. Insert Data
print("🚀 Indexing with OpenAI...")
processed_chunks = [
    {
        "text": chunk.text,
        "metadata": {
            "filename": chunk.meta.origin.filename if chunk.meta.origin else "doc.pdf",
            "page_numbers": [
                page_no
                for page_no in sorted(
                    set(
                        prov.page_no
                        for item in chunk.meta.doc_items
                        for prov in item.prov
                    )
                )
            ]
            or None,
            "title": chunk.meta.headings[0] if chunk.meta.headings else None,
        },
    }
    for chunk in chunks
]

table.add(processed_chunks)

# 8. Verify
print(f"🎉 Done! Rows indexed: {table.count_rows()}")
print(table.to_pandas().head(2))

[INFO] 2025-11-22 22:19:36,841 [RapidOCR] base.py:22: Using engine_name: torch


⏳ Converting PDF...


[INFO] 2025-11-22 22:19:36,916 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-11-22 22:19:36,917 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-11-22 22:19:37,330 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-22 22:19:37,335 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-11-22 22:19:37,337 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-11-22 22:19:37,460 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-22 22:19:37,543 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-11-22 22:19:3

🔪 Chunking...
✅ Created 21 chunks.
🚀 Indexing with OpenAI...
🎉 Done! Rows indexed: 21
                                                text  \
0  Christoph Auer Maksym Lysak Ahmed Nassar Miche...   
1  This technical report introduces Docling , an ...   

                                              vector  \
0  [0.005165622, -0.006144963, -0.021063859, 0.02...   
1  [0.0008674973, 0.0135612255, -0.024147633, -0....   

                                            metadata  
0  {'filename': '2408.09869v5.pdf', 'page_numbers...  
1  {'filename': '2408.09869v5.pdf', 'page_numbers...  


In [15]:
import lancedb

# --------------------------------------------------------------
# Connect to the database
# --------------------------------------------------------------

uri = "data/lancedb"
db = lancedb.connect(uri)


# --------------------------------------------------------------
# Load the table
# --------------------------------------------------------------

table = db.open_table("docling")


# --------------------------------------------------------------
# Search the table
# --------------------------------------------------------------

result = table.search(query="газобетоннa", query_type="vector").limit(3)
result.to_pandas()

,text,vector,metadata,_distance
0,Настоящата инструкция регламентира изискваният...,"[0.027452853, -0.021154394, -0.0029410352, 0.0...",{'filename': 'BOFI 50014.1 Приготвяне и запълв...,0.948265
1,Приготвянето на газобетонната смес включва сле...,"[0.034642067, -0.034101136, -0.009550798, 0.00...",{'filename': 'BOFI 50014.1 Приготвяне и запълв...,0.953884
2,"Производствения участък, където се изпълнява п...","[0.035472944, -0.0016953981, -0.006564234, 0.0...",{'filename': 'BOFI 50014.1 Приготвяне и запълв...,0.980890


In [9]:
!zip -r download.zip data

  adding: data/ (stored 0%)
  adding: data/lancedb/ (stored 0%)
  adding: data/lancedb/docling.lance/ (stored 0%)
  adding: data/lancedb/docling.lance/data/ (stored 0%)
  adding: data/lancedb/docling.lance/data/10110010000100111110001155b97640cda0da2bfc6b1749c6.lance (deflated 25%)
  adding: data/lancedb/docling.lance/_transactions/ (stored 0%)
  adding: data/lancedb/docling.lance/_transactions/0-021bd0ab-4b76-4b9d-8ea6-46e168351cb1.txn (deflated 36%)
  adding: data/lancedb/docling.lance/_transactions/1-43110221-7122-453a-8145-69687af7b38b.txn (deflated 9%)
  adding: data/lancedb/docling.lance/_versions/ (stored 0%)
  adding: data/lancedb/docling.lance/_versions/1.manifest (deflated 36%)
  adding: data/lancedb/docling.lance/_versions/2.manifest (deflated 35%)


In [16]:
import os
import shutil
import time
import glob
from typing import Dict, List, Tuple

import lancedb
import tiktoken
from docling.chunking import HybridChunker
from docling.document_converter import DocumentConverter
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector
from openai import OpenAI
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

# --- CONFIGURATION ---
OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

INPUT_FOLDER = "INPUT"
DB_PATH = "data/lancedb"
MAX_TOKENS = 8191

# Initialize OpenAI client
client = OpenAI()

# --- TOKENIZER WRAPPER ---
class OpenAITokenizerWrapper(PreTrainedTokenizerBase):
    def __init__(self, model_name: str = "cl100k_base", max_length: int = 8191, **kwargs):
        super().__init__(model_max_length=max_length, **kwargs)
        self.tokenizer = tiktoken.get_encoding(model_name)
        self._vocab_size = self.tokenizer.max_token_value

    def tokenize(self, text: str, **kwargs) -> List[str]:
        return [str(t) for t in self.tokenizer.encode(text)]

    def _tokenize(self, text: str) -> List[str]:
        return self.tokenize(text)

    def _convert_token_to_id(self, token: str) -> int:
        return int(token)

    def _convert_id_to_token(self, index: int) -> str:
        return str(index)

    def __len__(self) -> int:
        return self._vocab_size

    def get_vocab(self) -> Dict[str, int]:
        return dict(enumerate(range(self.vocab_size)))

    @property
    def vocab_size(self) -> int:
        return self._vocab_size

    def save_vocabulary(self, *args) -> Tuple[str]:
        return ()

    @classmethod
    def from_pretrained(cls, *args, **kwargs):
        return cls()

# --- SCHEMA SETUP ---
func = get_registry().get("openai").create(name="text-embedding-3-large")

class ChunkMetadata(LanceModel):
    filename: str | None
    page_numbers: List[int] | None
    title: str | None

class Chunks(LanceModel):
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()
    metadata: ChunkMetadata

# --- HELPER FUNCTIONS ---

def get_or_create_table():
    """Свързва се с базата."""
    db = lancedb.connect(DB_PATH)

    if "docling" in db.table_names():
        return db.open_table("docling")
    else:
        print("🆕 Създаване на нова таблица 'docling'...")
        return db.create_table("docling", schema=Chunks)

def process_single_file(file_path, table, tokenizer):
    filename = os.path.basename(file_path)
    print(f"\n📄 Обработка на: {filename}")

    # --- DEDUPLICATION CHECK ---
    # Проверяваме дали файлът вече съществува
    existing_files = table.search().where(f"metadata.filename = '{filename}'").limit(1).to_pandas()

    if not existing_files.empty:
        print(f"   🔄 Файлът '{filename}' вече съществува. Изтриване на старите записи...")
        table.delete(f"metadata.filename = '{filename}'")
        print("   🗑️ Старите записи са изтрити. Добавяне на новата версия...")
    # ---------------------------

    try:
        # 1. Docling Convert
        converter = DocumentConverter()
        result = converter.convert(file_path)

        # 2. Chunking
        chunker = HybridChunker(
            tokenizer=tokenizer,
            max_tokens=MAX_TOKENS,
            merge_peers=True,
        )
        chunks = list(chunker.chunk(dl_doc=result.document))
        print(f"   🧩 Генерирани {len(chunks)} парчета.")

        if not chunks:
            print("   ⚠️ Файлът е празен или не може да се прочете.")
            return False

        # 3. Prepare Data
        processed_chunks = [
            {
                "text": chunk.text,
                "metadata": {
                    "filename": filename,
                    "page_numbers": [
                        page_no for item in chunk.meta.doc_items for item_prov in item.prov for page_no in [item_prov.page_no]
                    ] or None,
                    "title": chunk.meta.headings[0] if chunk.meta.headings else None,
                },
            }
            for chunk in chunks
        ]

        # 4. Insert into LanceDB
        print("   🚀 Indexing to LanceDB...")
        table.add(processed_chunks)
        print("   ✅ Успешно индексиран!")
        return True

    except Exception as e:
        print(f"   ❌ ГРЕШКА при обработката на файла: {e}")
        return False

# --- MAIN LOOP ---

def main():
    # 1. Setup
    if not os.path.exists(INPUT_FOLDER):
        os.makedirs(INPUT_FOLDER)
        print(f"📁 Създадена папка '{INPUT_FOLDER}'. Слагай PDF файлове вътре!")

    tokenizer = OpenAITokenizerWrapper(model_name="cl100k_base", max_length=8191)
    table = get_or_create_table()

    print(f"\n👀 Мониторинг на папка '{INPUT_FOLDER}'...")
    print("Натисни Ctrl+C за спиране.\n")

    try:
        while True:
            pdf_files = glob.glob(os.path.join(INPUT_FOLDER, "*.pdf"))

            if pdf_files:
                print(f"🔎 Намерени {len(pdf_files)} файла за обработка!")

                for file_path in pdf_files:
                    success = process_single_file(file_path, table, tokenizer)

                    if success:
                        try:
                            os.remove(file_path)
                            print(f"   🗑️ Изтрит от Input: {os.path.basename(file_path)}")
                        except OSError as e:
                            print(f"   ⚠️ Не мога да изтрия файла: {e}")
                    else:
                        new_name = file_path + ".failed"
                        os.rename(file_path, new_name)
                        print(f"   ⚠️ Преименуван на {os.path.basename(new_name)}")

            time.sleep(10)

    except KeyboardInterrupt:
        print("\n🛑 Спиране на скрипта. Довиждане!")

if __name__ == "__main__":
    main()


👀 Мониторинг на папка 'INPUT'...
Натисни Ctrl+C за спиране.


🛑 Спиране на скрипта. Довиждане!


In [18]:
import os
import shutil
import time
import glob
from typing import Dict, List, Tuple

import lancedb
import tiktoken
from docling.chunking import HybridChunker
from docling.document_converter import DocumentConverter
from lancedb.embeddings import get_registry
from lancedb.pydantic import LanceModel, Vector
from openai import OpenAI
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

# --- LOGGING & WARNINGS SUPPRESSION ---
import logging
import warnings
# Скриваме спама от Docling и RapidOCR
logging.getLogger("docling").setLevel(logging.ERROR)
logging.getLogger("docling.backend.msword_backend").setLevel(logging.ERROR) # За DOCX грешките
warnings.filterwarnings("ignore")

# --- CONFIGURATION ---
OPENAI_API_KEY = ""
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

INPUT_FOLDER = "INPUT"
DB_PATH = "data/lancedb"
MAX_TOKENS = 8191

# Initialize OpenAI client
client = OpenAI()

# --- TOKENIZER WRAPPER ---
class OpenAITokenizerWrapper(PreTrainedTokenizerBase):
    def __init__(self, model_name: str = "cl100k_base", max_length: int = 8191, **kwargs):
        super().__init__(model_max_length=max_length, **kwargs)
        self.tokenizer = tiktoken.get_encoding(model_name)
        self._vocab_size = self.tokenizer.max_token_value

    def tokenize(self, text: str, **kwargs) -> List[str]:
        return [str(t) for t in self.tokenizer.encode(text)]

    def _tokenize(self, text: str) -> List[str]:
        return self.tokenize(text)

    def _convert_token_to_id(self, token: str) -> int:
        return int(token)

    def _convert_id_to_token(self, index: int) -> str:
        return str(index)

    def __len__(self) -> int:
        return self._vocab_size

    def get_vocab(self) -> Dict[str, int]:
        return dict(enumerate(range(self.vocab_size)))

    @property
    def vocab_size(self) -> int:
        return self._vocab_size

    def save_vocabulary(self, *args) -> Tuple[str]:
        return ()

    @classmethod
    def from_pretrained(cls, *args, **kwargs):
        return cls()

# --- SCHEMA SETUP ---
func = get_registry().get("openai").create(name="text-embedding-3-large")

class ChunkMetadata(LanceModel):
    filename: str | None
    # FIX: Премахваме "| None". LanceDB предпочита винаги List, дори да е празен.
    # Това оправя грешката "pyarrow.lib.DataType object has no attribute value_field"
    page_numbers: List[int]
    title: str | None

class Chunks(LanceModel):
    text: str = func.SourceField()
    vector: Vector(func.ndims()) = func.VectorField()
    metadata: ChunkMetadata

# --- HELPER FUNCTIONS ---

def get_or_create_table():
    """Свързва се с базата."""
    db = lancedb.connect(DB_PATH)

    if "docling" in db.table_names():
        return db.open_table("docling")
    else:
        print("🆕 Създаване на нова таблица 'docling'...")
        return db.create_table("docling", schema=Chunks)

def process_single_file(file_path, table, tokenizer):
    filename = os.path.basename(file_path)
    print(f"\n📄 Обработка на: {filename}")

    # --- DEDUPLICATION CHECK ---
    existing_files = table.search().where(f"metadata.filename = '{filename}'").limit(1).to_pandas()

    if not existing_files.empty:
        print(f"   🔄 Файлът '{filename}' вече съществува. Изтриване на старите записи...")
        table.delete(f"metadata.filename = '{filename}'")
        print("   🗑️ Старите записи са изтрити. Добавяне на новата версия...")
    # ---------------------------

    try:
        # 1. Docling Convert
        converter = DocumentConverter()
        result = converter.convert(file_path)

        # 2. Chunking
        chunker = HybridChunker(
            tokenizer=tokenizer,
            max_tokens=MAX_TOKENS,
            merge_peers=True,
        )
        chunks = list(chunker.chunk(dl_doc=result.document))
        print(f"   🧩 Генерирани {len(chunks)} парчета.")

        if not chunks:
            print("   ⚠️ Файлът е празен или не може да се прочете.")
            return False

        # 3. Prepare Data
        processed_chunks = [
            {
                "text": chunk.text,
                "metadata": {
                    "filename": filename,
                    "page_numbers": [
                        page_no for item in chunk.meta.doc_items for item_prov in item.prov for page_no in [item_prov.page_no]
                    ], # FIX: Махнахме "or None". Сега връща [] ако е празно.
                    "title": chunk.meta.headings[0] if chunk.meta.headings else "", # FIX: "" вместо None за по-сигурно
                },
            }
            for chunk in chunks
        ]

        # 4. Insert into LanceDB
        print("   🚀 Indexing to LanceDB...")
        table.add(processed_chunks)
        print("   ✅ Успешно индексиран!")
        return True

    except Exception as e:
        print(f"   ❌ ГРЕШКА при обработката на файла: {e}")
        return False

# --- MAIN LOOP ---

def main():
    if not os.path.exists(INPUT_FOLDER):
        os.makedirs(INPUT_FOLDER)
        print(f"📁 Създадена папка '{INPUT_FOLDER}'. Слагай файлове вътре (PDF, DOCX, PPTX, XLSX)!")

    tokenizer = OpenAITokenizerWrapper(model_name="cl100k_base", max_length=8191)
    table = get_or_create_table()

    print(f"\n👀 Мониторинг на папка '{INPUT_FOLDER}'...")
    print("Натисни Ctrl+C за спиране.\n")

    # Списък с поддържани формати
    extensions = ["*.pdf", "*.docx", "*.pptx", "*.html", "*.md", "*.xlsx"]

    try:
        while True:
            files_to_process = []
            for ext in extensions:
                files_to_process.extend(glob.glob(os.path.join(INPUT_FOLDER, ext)))

            if files_to_process:
                print(f"🔎 Намерени {len(files_to_process)} файла за обработка!")

                for file_path in files_to_process:
                    success = process_single_file(file_path, table, tokenizer)

                    if success:
                        try:
                            os.remove(file_path)
                            print(f"   🗑️ Изтрит от Input: {os.path.basename(file_path)}")
                        except OSError as e:
                            print(f"   ⚠️ Не мога да изтрия файла: {e}")
                    else:
                        new_name = file_path + ".failed"
                        try:
                            os.rename(file_path, new_name)
                            print(f"   ⚠️ Преименуван на {os.path.basename(new_name)}")
                        except:
                            pass # Ако и преименуването не стане, просто продължаваме

            time.sleep(10)

    except KeyboardInterrupt:
        print("\n🛑 Спиране на скрипта. Довиждане!")

if __name__ == "__main__":
    main()

📁 Създадена папка 'INPUT'. Слагай файлове вътре (PDF, DOCX, PPTX, XLSX)!

👀 Мониторинг на папка 'INPUT'...
Натисни Ctrl+C за спиране.



[INFO] 2025-11-22 23:00:27,930 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-22 23:00:27,975 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth
[INFO] 2025-11-22 23:00:27,976 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_det_infer.pth


🔎 Намерени 1 файла за обработка!

📄 Обработка на: BOFI 50014.1 Приготвяне и запълване на изделията с газобетоннa смес(3).pdf
   🔄 Файлът 'BOFI 50014.1 Приготвяне и запълване на изделията с газобетоннa смес(3).pdf' вече съществува. Изтриване на старите записи...
   🗑️ Старите записи са изтрити. Добавяне на новата версия...


[INFO] 2025-11-22 23:00:28,219 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-22 23:00:28,223 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-11-22 23:00:28,225 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_ptocr_mobile_v2.0_cls_infer.pth
[INFO] 2025-11-22 23:00:28,336 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-11-22 23:00:28,416 [RapidOCR] download_file.py:60: File exists and is valid: /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[INFO] 2025-11-22 23:00:28,417 [RapidOCR] torch.py:54: Using /usr/local/lib/python3.12/dist-packages/rapidocr/models/ch_PP-OCRv4_rec_infer.pth
[WARNING] 2025-11-22 23:00:33,653 [RapidOCR] main.py:123: The text detection result is empty
[WARNING] 2025-11-22 23:00:36,392 [RapidOCR] main.py:123: The text detection result is empty
[WARNING] 

   🧩 Генерирани 32 парчета.
   🚀 Indexing to LanceDB...
   ✅ Успешно индексиран!
   🗑️ Изтрит от Input: BOFI 50014.1 Приготвяне и запълване на изделията с газобетоннa смес(3).pdf
🔎 Намерени 1 файла за обработка!

📄 Обработка на: MOST  Стандарт расчета норм потребления материалов (неутв).docx
   🧩 Генерирани 11 парчета.
   🚀 Indexing to LanceDB...
   ✅ Успешно индексиран!
   🗑️ Изтрит от Input: MOST  Стандарт расчета норм потребления материалов (неутв).docx

🛑 Спиране на скрипта. Довиждане!
